In [1]:
from helpers.environment import Environment, EnvironmentType
from helpers.device import Device, Motor, AirConditioned, Light, LightType
from helpers.installation import Installation, InstallationType


environments = [
	Environment(
		"Escritório 1", 
		2.30, 
		3.9, 
		EnvironmentType.OFFICES_AND_CLASSROOMS, 
		Light(15, LightType.LED), 
		[AirConditioned("Ar condicionado escritório 1", 2000)]
	),

	Environment(
		"Escritório 2", 
		2.70, 
		5.10, 
		EnvironmentType.OFFICES_AND_CLASSROOMS, 
		Light(15, LightType.LED), 
		[AirConditioned("Ar condicionado escritório 2", 2000)]
	),

	Environment(
		"Recepção", 
		2.80, 
		5.10, 
		EnvironmentType.WAITING_ROOMS_AND_COMMUNAL_AREAS, 
		Light(15, LightType.LED), 
		[AirConditioned("Ar condicionado recepção", 2000)]
	),

	Environment(
		"Vestiário", 
		2.25, 
		2.70, 
		EnvironmentType.BATHROOMS_AND_OTHER_FACILITIES, 
		Light(15, LightType.LED), 
		[]
	),

	Environment(
		"Cozinha", 
		2.70, 
		1.90, 
		EnvironmentType.KITCHENS_AND_DINING_ROOMS, 
		Light(15, LightType.LED), 
		[Device("Microondas", 2000, 0.92), Device("Chaleira", 1200, 1.0)]
	),

	Environment(
		"Refeitório", 
		2.70, 
		2.90, 
		EnvironmentType.RESTAURANTS_CAFETERIAS_BARS_AND_HOTELS, 
		Light(15, LightType.LED), 
		[]
	),

	Environment(
		"Depósito", 
		2.70, 
		1.30, 
		EnvironmentType.WAREHOUSES_DEPOSITS_AND_STORAGE_AREAS, 
		Light(15, LightType.LED), 
		[]
	),

	Environment(
		"Banheiro", 
		1.95, 
		1.2, 
		EnvironmentType.BATHROOMS_AND_OTHER_FACILITIES, 
		Light(15, LightType.LED), 
		[Device("Chapinha / Secador", 1200, 1.0)]
	),

	Environment(
		"Banho", 
		1.99562, 
		1.99562, 
		EnvironmentType.BATHROOMS_AND_OTHER_FACILITIES, 
		Light(15, LightType.LED), 
		[Device("Chuveiro", 7000, 1.0), Device("Chapinha / Secador", 1200, 1.0)]
	),

	Environment(
		"Produção 1", 
		7.5, 
		8, 
		EnvironmentType.LAUNDRIES_AND_WORKSHOPS, 
		Light(15, LightType.LED), 
		[Motor(horse_power = 5.0, efficiency = 0.92, name = "Batedor de argila")] + 
		6 * [Motor(name = "Molde e cortador", horse_power = 1.0, power_factor = 0.85, efficiency = 0.95)]
	),

	Environment(
		"Produção 2", 
		7.5, 
		8, 
		EnvironmentType.LAUNDRIES_AND_WORKSHOPS, 
		Light(15, LightType.LED), 
		2 * [Motor(horse_power = 2.0, power_factor = 0.8, efficiency = 0.95)] + 
		[Motor(horse_power = 2.0, power_factor = 0.83, efficiency = 0.93)] +
		[Device(name = "Forno", power = 6000, power_factor = 1.0)]
	),

	Environment(
		"Produção 3", 
		4, 
		10.4, 
		EnvironmentType.LAUNDRIES_AND_WORKSHOPS, 
		Light(15, LightType.LED), 
		2 * [Motor(horse_power = 1.0, power_factor = 0.85, efficiency = 0.95)]
	),

	Environment(
		"Empacotamento", 
		4, 
		10.4, 
		EnvironmentType.LAUNDRIES_AND_WORKSHOPS, 
		Light(15, LightType.LED), 
		2 * [Motor(horse_power = 1.0, power_factor = 0.75, efficiency = 0.90)]
	),

]

installation = Installation(environments, InstallationType.THREE_PHASE)

print(installation)


Installation:
	Light Power: 4012.88W
	Light Number: 24
	Tug Power: 17000W
	Tug Number: 65
	Tue Power: 40266.0W
	Tue Number: 23
	Total Demanded Power: 20746.83W
	Installation Type: InstallationType.THREE_PHASE
	Phases: 3
	Total Demanded Apparent Power: 21838.77VA
	Total Demanded Current: 31.43A
	Input Branch Current: 39.29A
Enviroment: 
	Name: Escritório 1 - Area: 8.97m² - Perimeter: 12.4m
	Light: 
		Light: Lampada - 15 W - PF: 0.65 - 23.08 VA
		Recommended Power: 107.64W
		Minimum Light Power: 100W
		Light Power: 107.64W
		Minimum Light Points: 4
	Tug: 
		Minimum Tug Number: 4
		Minimum Tug Power: 400W
	Tue: 
		Minimum Tue Number: 1
Enviroment: 
	Name: Escritório 2 - Area: 13.77m² - Perimeter: 15.6m
	Light: 
		Light: Lampada - 15 W - PF: 0.65 - 23.08 VA
		Recommended Power: 165.24W
		Minimum Light Power: 160W
		Light Power: 165.24W
		Minimum Light Points: 4
	Tug: 
		Minimum Tug Number: 5
		Minimum Tug Power: 500W
	Tue: 
		Minimum Tue Number: 1
Enviroment: 
	Name: Recepção - Area: 14.28